In [107]:
# Data import using Spark ML
from pyspark.sql import SQLContext
#from pyspark.sql.types import *
import pandas as pd
from pyspark.ml import Pipeline
#from pyspark.ml.feature import StringIndexer, VectorIndexer

In [108]:
f = pd.read_csv("daily_weather.csv", sep=',', index_col=0)

# How many samples ?
f.shape

(1095, 10)

In [109]:
del f['relative_humidity_9am']

In [110]:
def functionG(row):
    if row['relative_humidity_3pm'] < 25:
        val = 1
    else:
        val = 0
    return val

In [111]:
# Convert to Format (label, features)
#g = f.copy(deep=True)
#low_humidity = pd.DataFrame()

f['label'] = f.apply(functionG, axis=1)

#How many samples are in Class 1 (low humidity) ?
f['label'].sum()

548

In [112]:
f.describe()

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_3pm,label
count,1092.000000,1091.000000,1091.000000,1090.000000,1094.000000,1093.000000,1089.000000,1091.000000,1095.000000,1095.000000
mean,918.745860,64.658066,142.065256,5.647467,148.813945,7.212678,3.218741,469.232765,35.635732,0.500457
std,3.204373,11.490511,69.247277,4.677773,67.921239,5.772890,37.516703,4613.974960,23.179102,0.500228
min,907.990000,34.790000,15.500000,0.693451,28.900000,1.185578,0.000000,0.000000,5.300000,0.000000
25%,916.450000,56.662304,64.900000,2.304048,75.897329,3.167080,0.000000,0.000000,16.993620,0.000000
50%,918.890000,65.372000,165.296258,3.914645,177.050000,5.018724,0.000000,0.000000,24.380000,1.000000
75%,921.055433,73.292000,190.800000,7.714985,201.183418,9.416251,0.000000,0.000000,52.790000,1.000000
max,927.610000,98.906000,333.000000,23.554978,314.700000,29.840780,655.334000,65487.000000,92.280000,1.000000


In [113]:
del f['relative_humidity_3pm']

In [114]:
f.columns[0:]
#temp = f.map(lambda line:LabeledPoint(line[0],[line[1:]]))

Index([u'air_pressure_9am', u'air_temp_9am', u'avg_wind_direction_9am',
       u'avg_wind_speed_9am', u'max_wind_direction_9am', u'max_wind_speed_9am',
       u'rain_accumulation_9am', u'rain_duration_9am', u'label'],
      dtype='object')

In [115]:
#from pyspark.mllib.linalg import DenseVector 
cols = ['air_pressure_9am','air_temp_9am','avg_wind_direction_9am','avg_wind_speed_9am','max_wind_direction_9am','max_wind_speed_9am','rain_accumulation_9am','rain_duration_9am']

In [116]:
#from pyspark.mllib.linalg import Vectors, VectorUDT

#w = pd.DataFrame()
#w['fea'] = f[cols].apply(lambda x:','.join(x.map(str)), axis=1)

#w['fea'] = [ f[c] for c in cols ]
#w.head(10)

In [117]:
from pyspark.ml.classification import DecisionTreeClassifier

In [118]:
#data = pd.concat([low_humidity, f], axis = 1)
#data.head(1)
data = f.copy(deep=True)
data.head(2)

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,label
0,921.551868,67.619839,197.965342,4.026048,210.834658,4.902075,1.020587e-14,0.0,1
1,923.100000,75.596000,178.300000,1.409272,205.200000,1.856660,3.069545e-13,0.0,1


In [119]:
sqlContext = SQLContext(sc)
dataDF = sqlContext.createDataFrame(data.dropna())
dataDF.take(2)

[Row(air_pressure_9am=921.5518680010001, air_temp_9am=67.6198391467, avg_wind_direction_9am=197.965342377, avg_wind_speed_9am=4.02604764663, max_wind_direction_9am=210.834657623, max_wind_speed_9am=4.9020753052399995, rain_accumulation_9am=1.0205866197799999e-14, rain_duration_9am=0.0, label=1),
 Row(air_pressure_9am=923.1, air_temp_9am=75.596, avg_wind_direction_9am=178.3, avg_wind_speed_9am=1.4092722, max_wind_direction_9am=205.2, max_wind_speed_9am=1.8566601999999999, rain_accumulation_9am=3.06954461848e-13, rain_duration_9am=0.0, label=1)]

# Features Column

In [120]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer
assembler = VectorAssembler(inputCols=cols, outputCol="features")
assembled = assembler.transform(dataDF)
assembled

DataFrame[air_pressure_9am: double, air_temp_9am: double, avg_wind_direction_9am: double, avg_wind_speed_9am: double, max_wind_direction_9am: double, max_wind_speed_9am: double, rain_accumulation_9am: double, rain_duration_9am: double, label: bigint, features: vector]

In [121]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(assembled)

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer= VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(assembled)

# Test and Training Data

In [122]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = assembled.randomSplit([0.7, 0.3], seed = 1234)

# Decision Tree in Spark

In [123]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures",
                  maxDepth=5, maxBins=32, minInstancesPerNode=20, minInfoGain=0.0,
                  maxMemoryInMB=256, cacheNodeIds=False, checkpointInterval=10,
                  impurity="gini")
    
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
# predictions.select("prediction", "indexedLabel", "features").show(5)

In [124]:
# Accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="precision")

accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g " % (accuracy))

Accuracy = 0.832237 


In [125]:
treeModel = model.stages[2]
# summary only
print(treeModel)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4b58bb9afac44f733c7f) of depth 5 with 39 nodes


# Naive Bayes in Spark

In [126]:
#Naive Bayes
from pyspark.ml.classification import NaiveBayes 
dtnb = NaiveBayes(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dtnb])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)
predictions.select("prediction", "indexedLabel", "features").show(5)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="precision")

accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g " % (accuracy))

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|[910.21,43.501999...|
|       0.0|         1.0|[910.34,50.72,174...|
|       0.0|         0.0|[911.0,86.432,202...|
|       0.0|         1.0|[912.9,78.332,167...|
|       0.0|         1.0|[914.13,58.388000...|
+----------+------------+--------------------+
only showing top 5 rows

Accuracy = 0.519737 


In [127]:
# Data cleaning (removing missing values) - check summary statistics

In [128]:
# Data Cleaning (replace with mean value) - check summary statistics

In [ ]:
# Summary Statistics


In [130]:
# Plots:

# Histogram
# Scatter Plot
# Bar Plot
# Box Plot


In [ ]:
# Data preparation (adding new columns)


In [ ]:
# Declaring Features and Target Class


In [ ]:
# Random Data splitting into test and train


In [ ]:
# Training the Decision Tree Classifier


In [ ]:
# Predicting 


In [ ]:
# Check Accuracy and Confusion Matrix of Decision Tree

In [ ]:
# Training the Naive Bayes Classifier


In [ ]:
# Predicting

In [ ]:
# Check Accuracy and Confusion Matrix of Decision Tree